In [1]:
import pandas as pd
import sympy as sp
import mpmath as mp 
import numpy as np
from scipy.fft import fft, ifft
from latex2sympy2 import latex2sympy, latex2latex
from manim import *
import calendar
import random
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import image
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, FFMpegWriter
from manim_voiceover import VoiceoverScene
from manim_voiceover.services.recorder import RecorderService
from manim_voiceover.services.gtts import GTTSService
import warnings
warnings.filterwarnings("ignore")
config.media_width = "100%"
config.verbosity = "WARNING"
import compass_functions as cf
from sympy import symbols, Eq, latex, sqrt,solve ,Function 
from sympy import   I, simplify, expand , print_latex,init_printing
from sympy import Symbol, sin,cos,arg,exp ,integrate,Derivative,Integral  
from sympy.utilities.lambdify import lambdify
sp.init_printing()
from IPython.display import Markdown as md
#DSP con placa de audio 
import sounddevice as sd
# Ajustes del fondo de plot para matplotlib 
plt.rcParams.update({
        "lines.color": "black",
        "patch.edgecolor": "black",
        "text.color": "white",
        "axes.facecolor": "black",
        "axes.edgecolor": "black",
        "axes.labelcolor": "black",
        "xtick.color": "white",
        "ytick.color": "white",
        "grid.color": "white",
        "figure.facecolor": "black",
        "figure.edgecolor": "black",
        "savefig.facecolor": "black",
        "savefig.edgecolor": "black"})



Manim Community v0.17.3

# Primera Escena

In [3]:
%%manim -qh TituloPortada
k_tiempo=2
esperar=3
Titulo=r"""
Si Ptolomeo  hubiese conocido a Fourier.

$$\zeta ^ t=  R_{_{_+}} z^{\omega t} +R_{_{_-}} z^{-\omega t}$$

"""
VoiceTitulo="Hola, bienvenidos al planeta matemático llamado Tau"
class TituloPortada(VoiceoverScene):
    def construct(self):
        self.set_speech_service(GTTSService(lang="es", tld="com",gender="male", accent="us"))
        tex = Tex(Titulo, font_size=70)
        with self.voiceover(text=VoiceTitulo) as tracker:
            self.play(Write(tex), run_time=tracker.duration*k_tiempo)

        #self.play(Write(tex))
        self.wait(esperar)

### Figuras de Lissajous

Si en un osciloscopio, que normalmente tiene dos canales de entrada que podemos llamar (X,Y) ponemos dos señales sinusoidales distintas, es equivalente a ver la gráfica definida en ecuaciones paramétricas de :

$$x(t)=A_x \sin(\omega_x t+\alpha)$$

$$y(t)=A_y \sin(\omega_y t+\beta)$$

La relación entre las fases de las dos expresiones anteriores se suele llamar :

$$\delta=\alpha-\beta$$


También podemos escribir:

$$x(t)=A_x \cos(\omega_x t+\alpha-\frac{\pi}{2})$$

$$y(t)=A_y \sin(\omega_y t+\beta)$$


### Usando exponenciales complejas

$$x(t) = \frac{A_x}{2i} \left(e^{i(\omega_x t + \alpha)} - e^{-i(\omega_x t + \alpha)}\right)$$

$$y(t) = \frac{A_y}{2i} \left(e^{i(\omega_y t + \beta)} - e^{-i(\omega_y t + \beta)}\right)$$

$$z(t)=\frac{A_x}{2i} \left(e^{i(\omega_x t + \alpha)} - e^{-i(\omega_x t + \alpha)}\right)+i\frac{A_y}{2i} \left(e^{i(\omega_y t + \beta)} - e^{-i(\omega_y t + \beta)}\right)$$

Simplificando y expandiendo podemos escribir:

$$\frac{1}{2} i A_x e^{-i(\omega_x t + \alpha)} - \frac{1}{2} i A_x e^{i(\omega_x t + \alpha)} - \frac{1}{2} A_y e^{-i(\omega_y t + \beta)} + \frac{1}{2} A_y e^{i(\omega_y t + \beta)}$$

$$\frac{1}{2} i A_x \left(e^{-i \omega_x t} e^{-i \alpha} \right) - \frac{1}{2} i A_x \left(e^{i \omega_x t} e^{i \alpha} \right) - \frac{1}{2} A_y \left(e^{-i \omega_y t} e^{-i \beta} \right) + \frac{1}{2} A_y \left(e^{i \omega_y t} e^{i \beta} \right)$$

## Intro

Ésta es una historia de ficción que tiene el objetivo de ayudar a ilustrar algunas intuiciones matemáticas que considero interesantes. 

La intención es que el contenido sea agradable para cualquier persona que no esté familiarizado con los temas tratados.  

Pero al mismo tiempo se agregarán detalles que pueden ser de interés para estudiantes de bachillerato o ya en nivel universitario. 

Tal vez sea de interés para docentes, ya que en la descripción hay un link a el código fuente usado para la realización de éste contenido usando SymPy, Manim, NumPy, Matplotlib, y varias librerías de Python adecuadas tanto para el cálculo y la manipulación simbólica sino también para la creación de contenido audiovisual en un workflow simplificado que me ha sido de utilidad ya que  soy nuevo en éste lenguaje y cómo seguro muchos viendo éste video vengo de la época en que se usaba Matlab. 


## Epiciclos, Atlas del tiempo y Números Compás
Para situarnos en éste experimento mental imaginaremos inicialmente un modelo simplificado de 3 cuerpos celestes tipo Estrella-Planeta-Satélite .(Sol-Tierra-Luna) 

Plantearemos los temas imaginando como serían presentados típicamente en un planeta ficticio (similar al nuestro)  que llamaremos Tau. 

En Tau los equivalentes a nuestros Ptolomeo / Copérnico /Kepler (En la historia de Tau tendremos uno solo que llamaremos simplemente Claudius) y el equivalente a Fourier (que llamaremos Joseph) fueron contemporáneos y amigos ambos estudiantes en la clase del profesor Hard Leon  (Cómo se llamaba en Tau el equivalente del gigante Leonhard Euler). 

En nuestro planeta Tierra, éstos personajes históricos junto con otros que mencionaremos con sus seudónimos  tales como Lissajous (En Tau llamado Antoine ) o J.S. Bach (en Tau lo llamaremos Sebastian), no fueron  contemporáneos (al menos no de la manera que ilustraremos en nuestro planeta ficticio). 

Por eso en nuestra historia paralela de Tau los pensaremos a-temporalmente sólo unidos por las relaciones entre sus descubrimientos individuales  y veremos cómo gracias a su gran profesor Hard Leon todos terminan descubriendo que están expresando lo mismo en diferentes lenguajes, mirando  y modelando la misma realidad fundamental desde distintas perspectivas. 


Podríamos haber elegido otros personajes históricos para hacer los paralelismos. Pero a medida que avancemos en el vídeo veremos por qué los elegidos son apropiados para dar una idea general y ayudar a la motivación/intuición  del público general y específicamente a estudiantes  que por primera vez se enfrentan a temas en la universidad que hacen uso de de las exponenciales complejas para, paradójicamente, simplificar los cálculos. 

También compartiré en la descripción un humilde documento con el código fuente usado para la creación de éste vídeo usando Python , y puntualmente se destaca el uso de la librería Manim creada por Grant S. y ahora mantenida por una enorme comunidad de desarrolladores que aprecian  la profunda  belleza de las clases magistrales  brindadas por éste gran divulgador.  

El documento también muestra como usar Sympy para manipulaciones simbólicas y transformaciones de expresiones matemáticas desde o hacia formato LaTeX y la creación de funciones NumPy equivalentes a las expresiones simbólicas  para poder hacer visualizaciones con Matplotlib. Todo desde una Jupyter Notebook que puede ser usada (con alguna modificación menor) sin problemas en Google Colab (ver links en la descripción ) 

Espero que los documentos compartidos sirvan también  a docentes para crear contenidos visuales y herramientas de análisis y cálculo complementarias a los temas que tratan en sus clases.  


## Números Compás 

Para dibujar la historia usaremos Números Complejos, pero justamente por que al usarlos todo resulta mucho mas fácil e intuitivo.

Por eso, no dejes de ver el video si no estás familiarizado con el tema, ya que espero que te quedes con intuiciones que te ayudarán a entenderlo luego. 

La idea original de éste vídeo era hacer una introducción básica a la idea de número bidimensional. Pero si entraba en todas las formalidades básicas en el formato típico  de la educación académica, nunca iba a poder terminar la historia que une todos los hilos. Es por eso que en la descripción habrá enlaces a introducciones formales recomendadas apropiadas para distintos niveles de familiaridad con los temas tratados. 

Imaginemos que en Tau a ésta idea de número de 2 dimensiones, al haber sido su línea histórica de descubrimientos muy distinta a la nuestra los terminaron llamando Números Compás. 

Éste nombre era apropiado, ya que  se sabía en la historia de Tau que Claudius usaba mucho el compás como instrumento de dibujo y medición y también usaba números bidimensionales en coordenadas polares y/o cartesianas que le indicaban en qué posición del papel representar cada instante/lugar de su modelo,se sabía que los usaba como notación para representar y aproximar curvas y analizar la mecánica celeste  de forma simplificada. 

En castellano, Compás  también es una palabra relacionada con la música mediante la idea de Ritmo  y ésto viene dado,como veremos, por la influencia del trabajo de Sebastian  en Tau.

Otro instrumento históricamente relacionado con el nombre elegido para los números mas populares en Tau, es el que usamos para ubicarnos en el espacio según el Campo Magnético de la Tierra que en castellano  llamamos Brújula (en inglés es Compass ).  

Y ésto era apropiado en Tau, ya que también tuvieron a un equivalente a J. C. Maxwell que describió la naturaleza ondulatoria del campo electromagnético inspirándose en intuiciones fundamentales de éste estilo. (Como ilustración de ésta relación, pensar cómo  es que el azúcar, al ser una molécula que tiene una determinada quiralidad, modifica el comportamiento de la luz  que pasa a través de un cilindro que contiene un líquido azucarado.(Ver  link en la descripción What is Wiggling ?)


## Qué relación tienen el calor, la luz y los epiciclos  ?

Entender cómo en Tau el trabajo de Joseph está íntimamente relacionado con el trabajo de Claudius llevará un poco de tiempo.  

Pero digamos como introducción que Joseph sufría mucho el frío (Según contó Steven S. en una entrevista con Grant S. ésto fue así en la Tierra, Fourier "tenía literalmente frío" ver link en la descripción). 

Es por ésto que Joseph en Tau,estaba buscando describir el comportamiento del calor usando el equivalente a  lo que en la Tierra llamamos ecuaciones diferenciales. 

Uno de los experimentos mentales de Joseph era unir dos barras metálicas una caliente y otra fría (o mejor dicho, menos caliente), y estudiar y tratar de modelar qué pasaba luego de un determinado tiempo $t$ con la distribución del calor entre las dos barras unidas. 

De forma misteriosa, inspirado por su profesor Hard Leon, ésto lo llevó a  aproximar una función que representada gráficamente tenía forma de escalón (ya que representaba un salto  entre las temperaturas de las dos barras en el punto de unión ). 

Las soluciones de esa ecuación diferencial, terminaron dando una solución que parecía funcionar usando la suma de "infinitas" combinaciones de ondas que tenían frecuencias que eran múltiplos enteros de una frecuencia fundamental que dependía de las longitudes de los materiales usados. (Ver fenómeno de Gibbs)


* PLOTS DE FENOMENO DE GIBBS

Claudius mientras tanto jugaba con  los cuerpos celestes usando modelos geocéntricos, heliocéntricos y otros. 


## Elipses y Simetrías 

Usó circunferencias inicialmente y luego se dió cuenta de que todo eran elipses de diferentes excentricidades, ya que todo lo que él era capaz de describir matemáticamente  veía que en esencia  estaba formado por la composición (o suma vectorial) de dos acciones/propiedades fundamentales opuestas  del tipo:

* (giro_sentido_anti_horario,giro_sentido_horario)

* $(+),(-)$ 

* (calor,frío)

* (luz,oscuridad) 

* (arriba, abajo)

* (derecha,izquierda)

* $(i,-i)$

* (1,-1)   



* Videos de elipses VARIOS (ver en documento complementario )


### Cómo contar en base dos es equivalente a obtener las 8 simetrías fundamentales de un número  $a+bi$ haciendo desplazamientos de $a$ y $b$ individualmente $180º$ y realizando permutación de ejes. 


| Binary | Decimal |    MSb     | LSb Symmetry 
|--------|---------|------------|------------- 
|b2 b1 b0| 0 to 7  | b2 Permute |  b1    b0
|--------|---------|------------|------------- 
| 000    | 0       | 0          |  a + i*b
| 001    | 1       | 0          |  a - i*b
| 010    | 2       | 0          | -a + i*b
| 011    | 3       | 0          | -a - i*b
| 100    | 4       | 1          |  b + i*a  
| 101    | 5       | 1          |  b - i*a  
| 110    | 6       | 1          | -b + i*a   
| 111    | 7       | 1          | -b - i*a  

## Sumando epiciclos para ajustar modelos

La obsesión  de Claudius era sumar epiciclos de forma tal que se puedan predecir mediante modelos matemáticos sencillos las trayectorias documentadas de los cuerpos celestes.

Pero al igual que el trabajo de Joseph, sus ideas llegaron mucho mas lejos y tuvieron aplicaciones en infinidad de áreas del conocimiento en la historia de Tau. 


## La noche de los descubrimientos - La Órbita Cuadrada y la desaparición del Sol

Los historiadores científicos de Tau cuentan que Joseph y Claudius tomaron nota  de las similitudes de sus trabajos cuando ambos en una noche larga de charla mientras estudiaban para la próxima clase con Hard Leon terminaron intentando calcular cuántas sub-lunas harían falta y con qué radios y períodos orbitales tendrían que girar una alrededor de la otra, para que sumando epiciclos se logre una órbita final anual en la que todo el año se vea  como un cuadrado.  


## ¿Qué le pasa a la trayectoria de una bola girando atada a un resorte si se suelta el resorte desde el centro? 

Si bien no cubriremos éste aspecto en profundidad en éste vídeo (ya que no soy físico y me gustaría que algún físico que vea ésto nos lo explique apropiadamente) vale mencionar que  esa noche, luego de unas cuantas cervezas, Joseph y Claudius postularon que que si el sol desapareciera  (perdiendo toda su masa) instantáneamente, ellos recién notarían el cambio en sus cuerpos luego de unos 8 minutos, durante los cuales el planeta seguiría girando normalmente en su órbita hasta salir disparado de forma similar a como saldría disparada una bola atada a un resorte que hacemos girar a una velocidad constante y luego soltamos el resorte .En Tau todo ésto a su vez tenía relación con el trabajo de  J. Clerk M. completado por Hermann M. y su curiosa introducción del tiempo como una coordenada más y luego gravitando todo hacia el trabajo de su despeinado discípulo Alberto ) (Ver links en la descripción)


## Figuras de Lissajous y el armonógrafo de Antoine 

Imaginaremos como Antoine  ayuda a Claudius y a Joseph a dibujar sus ideas usando ecuaciones paramétricas de las funciones usadas en sus modelos, con la ayuda de en un instrumento muy similar a lo que llamamos osciloscopio aquí en la Tierra.

* Plot Ejemplo paraméticas (ver en doc adjunto )


Podemos imaginar una especie de osciloscopio Steampunk, con propiedades especiales para darle un toque retro-futurista a la historia. Imaginemos que todas las visualizaciones que vendrán mas adelante, eran realizadas por ellos mientras compartían sus descubrimientos. 


## The Well-Tempered Clavier

Sebastian los inspira ese día logrando tocar 24 piezas musicales de idéntica construcción,pero en diferentes "tonalidades".  12 inspiradas progresivamente en  las horas mas luminosas del día y otras 12 para las horas menos luminosas (oscuras) del día. Un músico podrá apreciar la relación que tiene ésto con la escala menor y la escala mayor y las representaciones circulares de los símbolos musicales. 


## Calendario circular 

Veremos que, en Tau, usaban un calendario circular de resolución horaria. Cada hora estaba identificada también  por un color que se podía apreciar mirando a su estrella central con un instrumento apropiado. 


## El juego de Claudius - sumando Epiciclos 

Comenzando por el más pequeño de los cuerpos del sistema básico (E)strella-(P)laneta-(S)atélite definimos ingenuamente lo siguiente:

Si en el conjunto Tierra-Luna  el período en formato días-horas-minutos-segundos es $ T_l=29\space  d\space \space 12\space h\space \space 44\space m\space \space 2.9 \space s  $ en Tau supondremos que es exactamente un número entero apropiado para tener 12 meses de 30 días teniendo un año con un total exacto de 360 días (Podrás ir imaginando por  qué elegimos el  nombre del planeta como Tau  siendo $\tau=2\pi$) la longitud recorrida a lo largo de una vuelta completa ($360º$) en una circunferencia de radio $1u$ 

El período orbital del satélite  en Tau es entonces:

$$T_S=30d$$ 

Si en el sistema Tierra-Luna el radio orbital promedio de la luna es  $384,403 km$ en Tau tomaremos un camino un tanto sinuoso para definir su valor final, comencemos por definir algunas relaciones fundamentales:

Consideremos que queremos aproximar la circunferencia con un polígono regular de $N$ lados de longitud $L$ con $N$ proporcional a $T_S$ podemos pensar en una expresión para el radio orbital del (S)atélite respecto del 
(P)laneta:


$$R_{SP}=R_{_N}\cdot L $$ 

Donde $R_{_N}$ será un factor que nos asegurará que estemos formando un polígono  regular con N lados de longitud L .

Si dibujamos el cuadrado es fácil ver  por ejemplo  que para $N=4$ y $L=1$ formamos un cuadrado de perímetro $4 u$ y si centramos el cuadrado en el punto $(0,0)$ vemos que el radio del polígono termina siendo la hipotenusa de un triángulo rectángulo,con ángulo respecto a la horizontal de  de :

$$\frac{\pi}{4}=\frac{\tau}{8}$$ 

o expresado en  grados $45º$. 

Vemos que  ambos lados tienen longitud $\frac{1}{2}$ por lo que nos queda:

$$R=\sqrt{\left(\frac{1}{2}\right)^2+\left(\frac{1}{2}\right)^2}=\frac{1}{\sqrt{2}}=\frac{\sqrt{2}}{2}$$ 

Con un poco de trigonometría es fácil ver que para cualquier N podemos generalizar usando la siguiente expresión : 

$$R_N=\frac{1}{2 \cdot \sin{\left(\frac{\pi}{N}\right)}}=\frac{1}{2 \cdot \sin{\left(\frac{\tau}{2N}\right)}}$$



## Notación usada en Tau

En Tau usan un truco de notación similar al nuestro para las exponenciales complejas, pero basan a todas las
exponenciales complejas de módulo y argumento unitario con un número $\Theta=e^i$ y lo aprovechan para lograr 
expresiones más fáciles de leer. 

$$\Theta^{\theta}=e^{i\theta} = \cos(\theta) + i\sin(\theta)$$



### Aclaraciones para estudiantes 


Es importante recordar que cuando en matemáticas escribimos $e^{z}=e^{\alpha}\cdot e^{i\theta}$ estamos abusando de notación para representar una (auto)función,siendo $z  \in \mathbb{C}$ el argumento introducido en $exp(z)$ que está definida según su expansión en series de potencias . 

$$z = \alpha + i\theta $$

$$e^{z}=e^{\alpha+i\theta}=e^{\alpha}\cdot e^{i\theta}$$

$$e^z=exp(z)=\sum_{n=0}^{\infty} \frac{z^n}{n!} = 1 + \frac{z}{1!} + \frac{z^2}{2!} + \frac{z^3}{3!} + \cdots$$


$$e^{i\theta}= \sum_{n=0}^{\infty} \frac{(i \theta)^n}{n!} $$

$$e^{i\theta}=\sum_{n=0}^{\infty} \left[ \left(-1\right)^n \frac{\theta^{2n}}{(2n)!} \right] + i \sum_{n=0}^{\infty} \left[ \left(-1\right)^n \frac{\theta^{2n+1}}{(2n+1)!} \right]$$

Se puede expresar como un exponente ya que al evaluar ésta serie es fácil ver que es compatible con las siguientes reglas: 

$$x^a \cdot x^b=x^{a+b}$$

$$\frac{x^a}{x^b}=x^{a-b}$$

$$(x^a)^b=x^{ab}$$

$$\sqrt[a]{x}=x^{\frac{1}{a}}$$

Al igual que en la primer regla,el producto de dos exponenciales, resulta ser la exponencial de la suma de sus argumentos. 

$$exp(a)*exp(b)=exp(a+b)$$

Y así para el resto de las reglas

$$exp(a)/exp(b)=exp(a-b)$$


$$(exp(a))^b=exp(ab)$$

$$\sqrt[b]{exp(a)}=exp(\frac{a}{b})$$



### Un poco mas de detalle

Si $z=i \theta$ al evaluar la serie de potencias vemos que equivale a sucesivas rotaciones de $90º$  que nos dejan con la verdadera máquina detrás de una expresión tan sencilla, que es: 

$$e^{i\theta} = \sum_{n=0}^{\infty} \left[ \left(-1\right)^n \frac{\theta^{2n}}{(2n)!} \right] + i \sum_{n=0}^{\infty} \left[ \left(-1\right)^n \frac{\theta^{2n+1}}{(2n+1)!} \right]$$





### Explicación del ciclo mod 4 paso a paso 


Notar como al evaluar la expansión en serie de potencias siempre obtenemos un ciclo mod 4  antihorario del tipo:

$$1,i,-1,-i $$


Lo que en Tau se suele escribir:

$$\Theta^{\tau},\Theta^{\frac{\tau}{4}},\Theta^{\frac{\tau}{2}},\Theta^{\frac{3\tau}{4}}$$

o

$$\Theta^{2\pi},\Theta^{\frac{\pi}{2}},\Theta^{\pi},\Theta^{\frac{3\pi}{2}}$$



Recordando la expresión general de Taylor: 

$$f(x) = f(a) + f'(a)(x - a) + \frac{{f''(a)}}{{2!}}(x - a)^2 + \frac{{f'''(a)}}{{3!}}(x - a)^3 + \ldots$$

$$f(x) = exp(ix)$$ 

$$x=\theta$$ 

$$a=0$$ 

Evaluando para los primeros 5 valores de n, nos damos cuenta del comportamiento cíclico debido a que la derivada
de $\frac{d}{dt}e^{it}=ie^{it}$ usando la regla de la cadena, por lo cual es inmediato ver que :

$$e^{i\theta}= \sum_{n=0}^{\infty} \frac{(i \theta)^n}{n!} $$

Evaluando los términos de la anterior sumatoria tenemos:

(n = 0):
$$ \frac{{(i\theta)^0}}{{0!}} = 1 $$

(n = 1):
$$ \frac{{(i\theta)^1}}{{1!}} = i\theta $$

(n = 2):
$$ \frac{{(i\theta)^2}}{{2!}} = -\frac{{\theta^2}}{{2}} $$

(n = 3):
$$ \frac{{(i\theta)^3}}{{3!}} = -\frac{{i\theta^3}}{{6}} $$

(n = 4):
$$ \frac{{(i\theta)^4}}{{4!}} = \frac{{\theta^4}}{{24}} $$

Combinando los términos reales e imaginarios tenemos:

$$ \exp(i\theta) = 1 + i\theta - \frac{{\theta^2}}{{2}} - \frac{{i\theta^3}}{{6}} + \frac{{\theta^4}}{{24}} - \ldots $$



$$e^{i \theta} = \sum_{n=0}^{\infty} \frac{(i \theta)^n}{n!} $$

* Se puede recordar siempre que el coseno es par y el seno es impar para recordar estas expresiones que se pueden obtener simplemente derivando sucesivamente el coseno y el seno según su serie de Taylor (o MCLaurin) como hicimos para la exponencial y veremos ésto claramente.

$$\cos(\theta) = \sum_{n=0}^{\infty} (-1)^n \frac{\theta^{2n}}{(2n)!} = 1 - \frac{\theta^2}{2!} + \frac{\theta^4}{4!} - \frac{\theta^6}{6!} + \cdots$$

$$\sin(\theta) = \sum_{n=0}^{\infty} (-1)^n \frac{\theta^{2n+1}}{(2n+1)!} = \theta - \frac{\theta^3}{3!} + \frac{\theta^5}{5!} - \frac{\theta^7}{7!} + \cdots$$

## Motivación para explorar los números compás en Tau

$$
\begin{align*}
x[n] &= \sum_{k=0}^{N-1}c_k e^{i \omega k n} \\
c_k &= \frac{1}{N}\sum_{n=0}^{N-1}x[n]e^{-i \omega k n}
\end{align*}
$$


$$
\begin{align*}
x(t) = \sum_{n=-\infty}^{\infty}c_n e^{i \omega nt} \\
c_n &= \int_{-\frac{P}{2}}^{\frac{P}{2}}x(t)e^{-i \omega nt}dt
\end{align*}
$$
$$\omega = \frac{2\pi}{P}$$

### Atlas del tiempo - Calendario circular en Tau 

Descripción de variables usadas en Tau para describir el calendario horario :

* a para los años

* m para los meses

* d para los días

* h para las horas


$$\hat a=r_a\space \Theta^{\omega_a a }$$

$$\hat m=r_m\space \Theta^{\omega_m m }$$

$$\hat d=r_d \space \Theta^{\omega_d d }$$

$$\hat h=r_h\space \Theta^{\omega_h h }$$


Siendo 

$$\omega_a=\frac{\tau}{T_a}$$

$$\omega_m=\frac{\tau}{T_m}$$

$$\omega_d=\frac{\tau}{T_d}$$

$$\omega_h=\frac{\tau}{T_h}$$




Como vimos 

En Tau llaman  Atlas del Tiempo  a la gráfica paramétrica obtenida combinando como epiciclos las expresiones anteriores: 

$$\hat A = \hat a+\hat m+\hat d+\hat h $$